In [1]:
import pandas as pd
import numpy as np

## load and merge Clinica BIDS participant files and t1/flair/fmri path files

In these steps, we clean up the participant and image modality files created by Clinica. 

We ran Clinica in a subject-specific manner (one subject at a time) because this really sped up the process. But after we ran Clinica on all subjects, we need to merge their BIDS output directories and files into one directory and file. 

In this process of merging subjects into one big file, I forgot the drop the header when merging them, so below I drop all rows that contain the header (column names). 

In [4]:
from pathlib import Path
import pandas as pd

# Set this to your BIDS root
bids_root = Path("/N/project/statadni/20250922_Saige/adni_db/bids/participants/")

all_dfs = []
missing_subjects = []

# Loop over all subject directories
for sub_dir in sorted(p for p in bids_root.iterdir() if p.is_dir() and p.name.startswith("sub-")):
    sessions_file = sub_dir / f"{sub_dir.name}_sessions.tsv"

    if not sessions_file.exists():
        # sessions file missing for this subject
        missing_subjects.append(sub_dir.name)
        continue

    try:
        df = pd.read_csv(sessions_file, sep="\t")

        # Add subject identifier if not already present
        if "participant_id" not in df.columns and "subject" not in df.columns:
            df["participant_id"] = sub_dir.name

        all_dfs.append(df)
    except Exception as e:
        print(f"Error reading {sessions_file}: {e}")

# Merge all dataframes (union of columns)
if all_dfs:
    merged = pd.concat(all_dfs, ignore_index=True, sort=True)

    out_file = bids_root / "all_subjects_sessions.tsv"
    merged.to_csv(out_file, sep="\t", index=False)

    print(f"Saved merged file to: {out_file}")
    print(f"Shape: {merged.shape[0]} rows × {merged.shape[1]} columns")
else:
    print("No sessions.tsv files were successfully read.")

# Report subjects that had no sessions file
if missing_subjects:
    print(f"\nSubjects missing *_sessions.tsv ({len(missing_subjects)}):")
    for sub in missing_subjects:
        print(sub)

Saved merged file to: /N/project/statadni/20250922_Saige/adni_db/bids/participants/all_subjects_sessions.tsv
Shape: 11588 rows × 786 columns

Subjects missing *_sessions.tsv (124):
sub-ADNI002S5256
sub-ADNI003S0907
sub-ADNI003S0981
sub-ADNI003S1057
sub-ADNI003S4555
sub-ADNI005S0546
sub-ADNI005S0553
sub-ADNI006S1130
sub-ADNI007S0101
sub-ADNI007S0698
sub-ADNI007S1206
sub-ADNI010S0420
sub-ADNI014S0169
sub-ADNI014S0519
sub-ADNI014S0520
sub-ADNI014S0563
sub-ADNI016S0702
sub-ADNI016S1117
sub-ADNI016S1326
sub-ADNI018S0055
sub-ADNI018S0142
sub-ADNI020S1288
sub-ADNI021S0159
sub-ADNI021S0276
sub-ADNI021S0984
sub-ADNI022S0096
sub-ADNI022S0130
sub-ADNI022S1097
sub-ADNI022S1351
sub-ADNI022S1394
sub-ADNI023S0042
sub-ADNI023S0058
sub-ADNI023S0061
sub-ADNI023S0126
sub-ADNI023S0217
sub-ADNI023S0331
sub-ADNI023S0376
sub-ADNI023S0887
sub-ADNI023S0926
sub-ADNI023S1046
sub-ADNI024S0985
sub-ADNI027S0116
sub-ADNI027S0307
sub-ADNI027S0408
sub-ADNI027S0644
sub-ADNI027S0835
sub-ADNI027S1045
sub-ADNI029S1318
sub

In [ ]:
from pathlib import Path
import pandas as pd

participants_root = Path("/N/project/statadni/20250922_Saige/adni_db/bids/participants")

# List of modality files to search for
modalities = {
    "fmri":  "fmri_paths.tsv",
    "flair": "flair_paths.tsv",
    "t1":    "t1_paths.tsv"
}

all_rows = []

def extract_sub_ses(tsv_path: Path):
    """Extract sub-XXX and ses-YYY from path."""
    # .../participants/sub-XXX/ses-YYY/conversion_invo/v0/file.tsv
    sub_id = tsv_path.parents[3].name   # sub-XXX
    ses_id = tsv_path.parents[2].name   # ses-YYY
    return sub_id, ses_id


for modality, filename in modalities.items():
    pattern = f"sub-*/ses-*/conversion_invo/v0/{filename}"

    for tsv_path in participants_root.glob(pattern):
        try:
            df = pd.read_csv(tsv_path, sep="\t")
        except Exception as e:
            print(f"Error reading {tsv_path}: {e}")
            continue

        # Add subject and session
        sub_id, ses_id = extract_sub_ses(tsv_path)
        df["participant_id"] = sub_id
        df["session_id"] = ses_id

        # Add modality column
        df["modality"] = modality

        all_rows.append(df)


# Concatenate all modalities + all subjects/sessions
if not all_rows:
    raise RuntimeError("No modality conversion files found.")

merged = pd.concat(all_rows, ignore_index=True, sort=True)

# Drop columns where all values are NaN
merged = merged.dropna(axis=1, how="all")

# Output file
out_file = participants_root / "all_modalities_paths.tsv"
merged.to_csv(out_file, sep="\t", index=False)

print(f"Saved merged file to {out_file}")
print(merged.shape)

In [ ]:
from pathlib import Path
import pandas as pd

participants_root = Path("/N/project/statadni/20250922_Saige/adni_db/bids/participants")

def extract_sub_ses(tsv_path: Path):
    """
    Extract sub-XXX and ses-YYY from path:
    .../participants/sub-XXX/ses-YYY/conversion_invo/v0/filename.tsv
    """
    sub_id = tsv_path.parents[3].name   # sub-XXX
    ses_id = tsv_path.parents[2].name   # ses-YYY
    return sub_id, ses_id


def merge_participants_tsv():
    pattern = "sub-*/ses-*/conversion_invo/v0/participants.tsv"
    all_dfs = []
    n_files = 0

    for tsv_path in participants_root.glob(pattern):
        n_files += 1
        try:
            df = pd.read_csv(tsv_path, sep="\t")
        except Exception as e:
            print(f"Error reading {tsv_path}: {e}")
            continue

        # Add subject / session
        sub_id, ses_id = extract_sub_ses(tsv_path)

        if "participant_id" not in df.columns:
            df["participant_id"] = sub_id
        if "session_id" not in df.columns and "session" not in df.columns:
            df["session_id"] = ses_id

        all_dfs.append(df)

    if not all_dfs:
        print("No participants.tsv files were successfully read.")
        return

    merged = pd.concat(all_dfs, ignore_index=True, sort=True)

    # Drop columns where ALL rows are NaN
    merged = merged.dropna(axis=1, how="all")

    out_file = participants_root / "all_participants.tsv"
    merged.to_csv(out_file, sep="\t", index=False)

    print(f"Merged {n_files} participants.tsv files into: {out_file}")
    print(f"Shape: {merged.shape[0]} rows × {merged.shape[1]} columns")


# Call the function
merge_participants_tsv()

In [2]:
# load the Clinica participants file.
participant_df = pd.read_csv('/N/project/statadni/20250922_Saige/adni_db/bids/metadata/participants.tsv', sep='\t')

In [24]:
# drop the header name rows.
participant_df = participant_df[participant_df.participant_id != 'participant_id']

In [25]:
# check the number of unique subjects.
len(participant_df['participant_id'].value_counts())

1689

In [26]:
participant_df.shape

(1689, 22)

In [27]:
participant_df.head()

,alternative_id_1,participant_id,sex,education_level,age_bl,marital_status,ethnic_cat,adni_rid,site,original_study,...,adas11_bl,adas13_bl,apoe4,apoe_gen1,apoe_gen2,ravlt_immediate_bl,moca_bl,trabscor_bl,mpaccdigit_bl,mpacctrails_bl
0,002_S_0295,sub-ADNI002S0295,M,18,84.8,Married,Not Hisp/Latino,295,2,ADNI1,...,3.00,4.00,1.0,3.0,4.0,56.0,NaN,300.0,-0.110553,-3.635890
1,002_S_0413,sub-ADNI002S0413,F,16,76.3,Married,Not Hisp/Latino,413,2,ADNI1,...,3.33,4.33,0.0,4.0,4.0,52.0,NaN,55.0,1.252420,1.268850
2,002_S_0685,sub-ADNI002S0685,F,16,89.6,Married,Not Hisp/Latino,685,2,ADNI1,...,3.67,6.67,0.0,NaN,NaN,36.0,NaN,67.0,0.625145,0.862282
3,002_S_0729,sub-ADNI002S0729,F,16,65.1,Married,Not Hisp/Latino,729,2,ADNI1,...,6.67,14.67,1.0,4.0,4.0,38.0,NaN,73.0,-8.768470,-8.341950
4,002_S_1155,sub-ADNI002S1155,M,20,57.8,Married,Not Hisp/Latino,1155,2,ADNI1,...,10.00,17.00,0.0,3.0,3.0,33.0,NaN,70.0,-5.355030,-5.613500


In [63]:
# save the file without the header name rows.
participant_df.to_csv('/N/project/statadni/20250922_Saige/adni_db/bids/metadata/participants.tsv', sep='\t', index=False)

In [28]:
# load the Clinica t1_paths file.
t1 = pd.read_csv('/N/project/statadni/20250922_Saige/adni_db/bids/participants/conversion_info/v0/t1_paths.tsv', sep='\t')

In [29]:
# drop the header name rows.
t1 = t1[t1.Subject_ID != 'Subject_ID']

In [30]:
# check the number of unique subjects.
len(t1['Subject_ID'].value_counts())

1838

In [31]:
# check the number of nifti files.
t1.shape

(15786, 12)

In [32]:
# drop the duplicate rows (probably due to a small amount of overlap between me & Vincent's ADNI3 downloads).
t1.drop_duplicates(inplace=True)

In [33]:
# Add T1 to column names for when I merge this with the fMRI & FLAIR image files.
t1.rename(columns={'Sequence':'T1_Sequence','Scan_Date':'T1_Scan_Date','Study_ID':'T1_Study_ID','Series_ID':'T1_Series_ID',
                    'Image_ID':'T1_Image_ID','Field_Strength':'T1_Field_Strength','Is_Dicom':'T1_Is_Dicom','Path':'T1_Path'},inplace=True)

In [70]:
# save the file without the header name rows.
t1.to_csv('/N/project/statadni/20250922_Saige/BIDS/t1_paths.tsv', sep='\t', index=False)

In [34]:
# load the t2_paths file.
flair = pd.read_csv('/N/project/statadni/20250922_Saige/adni_db/bids/participants/conversion_info/v0/flair_paths.tsv', sep='\t')

In [35]:
# drop the header name rows.
flair = flair[flair.Subject_ID != 'Subject_ID']

In [36]:
# drop the duplicate rows (probably due to a small amount of overlap between me & Vincent's ADNI3 downloads).
flair.drop_duplicates(inplace=True)

In [37]:
# check the number of nifti files.
flair.shape

(10079, 11)

In [38]:
# check the number of unique subjects.
len(flair['Subject_ID'].value_counts())

1687

In [64]:
# save the file without the header name rows.
flair.to_csv('/N/project/statadni/20250922_Saige/BIDS/flair_paths.tsv', sep='\t', index=False)

In [39]:
# load the fmri_paths file.
fmri = pd.read_csv('/N/project/statadni/20250922_Saige/adni_db/bids/participants/conversion_info/v0/fmri_paths.tsv', sep='\t')

In [40]:
# drop the header name rows.
fmri = fmri[fmri.Subject_ID != 'Subject_ID']

In [41]:
# drop the duplicate rows (probably due to a small amount of overlap between me & Vincent's ADNI3 downloads).
fmri.drop_duplicates(inplace=True)

In [42]:
# check the number of unique subjects.
len(fmri['Subject_ID'].value_counts())

1153

In [43]:
# check the number of nifti files.
fmri.shape

(4818, 11)

There are **4818 fmri nifti files** from **1153 unique subjects** that successfully ran through Clinica (DICOM to NIFTI conversion and BIDS organization).

In [58]:
# save the file without the header name rows.
fmri.to_csv('/N/project/statadni/20250922_Saige/BIDS/fmri_paths.tsv', sep='\t', index=False)

In [44]:
fmri.rename(columns={'Sequence':'fMRI_Sequence','Scan_Date':'fMRI_Scan_Date','Study_ID':'fMRI_Study_ID','Series_ID':'fMRI_Series_ID',
                    'Image_ID':'fMRI_Image_ID','Field_Strength':'fMRI_Field_Strength','Is_Dicom':'fMRI_Is_Dicom','Path':'fMRI_Path'},inplace=True)

In [45]:
# Merge together the fMRI and T1w paths files
fmri_t1 = pd.merge(fmri, t1, how='inner')

In [46]:
len(fmri_t1)

8434

In [47]:
len(fmri_t1['Subject_ID'].value_counts())

1151

There are **8434 T1w & rs-fMRI nifti files** from **1151 unique subjects** that successfully ran through Clinica (DICOM to NIFTI conversion and BIDS organization).

In [48]:
fmri_t1['T1_Sequence'].value_counts()

T1_Sequence
Accelerated_Sagittal_MPRAGE               3621
MT1__N3m                                  1790
Accelerated_Sag_IR-FSPGR                   960
Sagittal_3D_Accelerated_MPRAGE             950
Accelerated_Sagittal_IR-FSPGR              798
Accelerated_Sagittal_MPRAGE_Phase_A-P      110
Sagittal_3D_Accelerated_0_angle_MPRAGE      78
MT1__GradWarp__N3m                          43
Sag_Accel_IR-FSPGR                          35
ORIG_Accelerated_Sag_IR-FSPGR               19
MPRAGE                                      10
REPEAT_Accelerated_Sagittal_MPRAGE           4
Sag_IR-SPGR                                  4
Accelerated_Sagittal_MPRAGE_L__R             4
Accelerated_Satittal_MPRAGE                  4
Accelerated_Sagittal_MPRAGE_MPR_Cor          4
Name: count, dtype: int64

In [90]:
t1_names = ['Accelerated_Sagittal_MPRAGE',
'MT1__N3m ',
'Accelerated_Sag_IR-FSPGR',
'Sagittal_3D_Accelerated_MPRAGE',
'Accelerated_Sagittal_IR-FSPGR',
'Accelerated_Sagittal_MPRAGE_Phase_A-P',
'Sagittal_3D_Accelerated_0_angle_MPRAGE',
'MT1__GradWarp__N3m',
'Sag_Accel_IR-FSPGR',
'ORIG_Accelerated_Sag_IR-FSPGR',
'MPRAGE',
'REPEAT_Accelerated_Sagittal_MPRAGE',
'Sag_IR-SPGR',
'Accelerated_Sagittal_MPRAGE_L__R',
'Accelerated_Satittal_MPRAGE',
'Accelerated_Sagittal_MPRAGE_MPR_Cor']

In [49]:
fmri_t1.head()

,Subject_ID,VISCODE,Visit,fMRI_Sequence,fMRI_Scan_Date,fMRI_Study_ID,fMRI_Series_ID,fMRI_Image_ID,fMRI_Field_Strength,fMRI_Is_Dicom,fMRI_Path,T1_Sequence,T1_Scan_Date,T1_Study_ID,T1_Series_ID,T1_Image_ID,T1_Field_Strength,Original,T1_Is_Dicom,T1_Path
0,002_S_0413,m60,ADNI2 Initial Visit-Cont Pt,Resting_State_fMRI,2011-06-16,34751,111991,240811,NaN,True,/N/project/statadni/20231212_ADR012021_UtahBac...,MT1__N3m,2011-06-16,34751,111989,242896,3.0,False,False,/N/project/statadni/20231212_ADR012021_UtahBac...
1,002_S_0413,m60,ADNI2 Initial Visit-Cont Pt,Resting_State_fMRI,2011-06-16,34751,111991,240811,NaN,True,/N/project/statadni/20231212_ADR012021_UtahBac...,MT1__N3m,2011-06-16,34751,111989,242896,3.0,False,True,NaN
2,002_S_0413,m72,ADNI2 Year 1 Visit,Resting_State_fMRI,2012-05-15,45738,150694,304790,NaN,True,/N/project/statadni/20231212_ADR012021_UtahBac...,MT1__N3m,2012-05-15,45738,150696,312703,3.0,False,False,/N/project/statadni/20231212_ADR012021_UtahBac...
3,002_S_0413,m72,ADNI2 Year 1 Visit,Resting_State_fMRI,2012-05-15,45738,150694,304790,NaN,True,/N/project/statadni/20231212_ADR012021_UtahBac...,MT1__N3m,2012-05-15,45738,150696,312703,3.0,False,True,NaN
4,002_S_0413,m84,ADNI2 Year 2 Visit,Resting_State_fMRI,2013-05-10,58370,189129,371994,NaN,True,/N/project/statadni/20231212_ADR012021_UtahBac...,MT1__N3m,2013-05-10,58370,189127,373133,3.0,False,False,/N/project/statadni/20231212_ADR012021_UtahBac...


In [50]:
fmri_t1['fMRI_Sequence'].value_counts()

fMRI_Sequence
Axial_rsfMRI__Eyes_Open_                           2821
Resting_State_fMRI                                 1425
Axial_MB_rsfMRI__Eyes_Open_                        1229
Axial_fcMRI__EYES_OPEN_                             913
Axial_rsfMRI__EYES_OPEN_                            806
Axial_fcMRI__Eyes_Open_                             496
Extended_Resting_State_fMRI                         371
Axial_rsfMRI__Eyes_Open__-phase_P_to_A              110
Axial_fcMRI_0_angle__EYES_OPEN_                      78
Axial_RESTING_fcMRI__EYES_OPEN_                      41
Axial_fcMRI                                          40
Axial_MB_rsfMRI__Eyes_Open____straight_no_angle      39
Axial_rsfMRI__Eyes_Open__10_min__-PJ                 19
Axial_rsfMRI__Eyes_Open__Phase_Direction_P_A         14
Axial_rsFMRI_Eyes_Open                                8
Axial_-_Advanced_fMRI_64_Channel                      4
Extended_Resting_State_fMRI_CLEAR                     4
Extended_AXIAL_rsfMRI_EYES_OPEN   

## BIDS-ify the recovered T1w dicoms (where I manually ran dcm2niix successfully)

I manually went through the missing_T1.csv and checked the dicom directories to verify if the T1w DICOMS exist for the scan date (that matches the scan date of the resting state scan). 

I will try to script this process, but due to the inconsistent directory names, it is difficult. 

Out of the 76 subjects who were missing a T1w image (but had a resting-state scan), I could re-run dcm2niix manually myself on 49 subjects. 

These subjects need to be converted into BIDS format and moved from the DICOM directories into the BIDS directory. We also need to add a row to the t1_paths.tsv file for these subjects. 

In [65]:
m_t1 = pd.read_csv('/N/project/statadni/20250922_Saige/QC/missing_t1_saved_BIDSify.csv')

In [66]:
m_t1.shape

(49, 6)

In [67]:
m_t1.columns

Index(['Image_ID', 'Subject_ID', 'VISCODE', 'Path', 'JSON_path', 'Notes'], dtype='object')

In [144]:
m_t1_path = m_t1['Path'].str.split(pat="/", expand=True)

In [146]:
import glob

In [158]:
! ls /N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/ADNI3_MRI_fMRI_M/114_S_6039/*/*/*.nii

/N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/ADNI3_MRI_fMRI_M/114_S_6039/Accelerated_Sagittal_MPRAGE/2017-07-21_12_13_00.0/2017-07-21_12_13_00.0_0_3.nii


In [168]:
m_t1.head()

,Image_ID,Subject_ID,VISCODE,Path,JSON_path,Notes
0,879211,114_S_6039,bl,/N/project/statadni/20231212_ADR012021_UtahBac...,/N/project/statadni/20250922_Saige/adni_db/bid...,NaN
1,896824,941_S_4365,m66,/N/project/statadni/20231212_ADR012021_UtahBac...,/N/project/statadni/20250922_Saige/adni_db/bid...,NaN
2,223896,002_S_1261,m48,/N/project/statadni/20231212_ADR012021_UtahBac...,/N/project/statadni/20250922_Saige/adni_db/bid...,NaN
3,233437,002_S_1280,m48,/N/project/statadni/20231212_ADR012021_UtahBac...,/N/project/statadni/20250922_Saige/adni_db/bid...,NaN
4,180734,002_S_2010,bl,/N/project/statadni/20231212_ADR012021_UtahBac...,/N/project/statadni/20250922_Saige/adni_db/bid...,NaN


In [219]:
data_nii = []
for i, row in m_t1_path.iterrows():
    path1 = "/N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/" 
    path2 = m_t1_path.iloc[i][7] + "/" 
    path3 = m_t1_path.iloc[i][8] + "/"
    path4 = "/" + m_t1_path.loc[i][10] + "/"
    for j, name in enumerate(t1_names):
        path = path1 + path2 + path3 + name + path4 + "*.nii"
        path_go = path1 + path2 + path3 + name + "/*" + m_t1_path.loc[i][10].split(".")[0].replace("-","").replace("_","") + "*.nii"
        if glob.glob(path):
            print(path)
            data_nii.append(path)
        if glob.glob(path_go):
            print(path_go)
            data_nii.append(path_go)

/N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/ADNI3_MRI_fMRI_M/114_S_6039/Accelerated_Sagittal_MPRAGE/2017-07-21_12_13_00.0/*.nii
/N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/ADNI3_MRI_fMRI_M/941_S_4365/Accelerated_Sagittal_MPRAGE/2017-08-28_14_06_46.0/*.nii
/N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/ADNIGO_MRI_fMRI_F/002_S_1261/MPRAGE/*20110314160431*.nii
/N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/ADNIGO_MRI_fMRI_F/002_S_1280/MPRAGE/*20110504132634*.nii
/N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/ADNIGO_MRI_fMRI_F/002_S_2010/MPRAGE/*20100624142128*.nii
/N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/ADNIGO_MRI_fMRI_F/002_S_2010/MPRAGE/*20101022153121*.nii
/N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/ADNIGO_MRI_fMRI_F/002_S_2010/MPRAGE/*20110122123155*.nii
/N/project/stat

In [220]:
df_nii = pd.DataFrame(data_nii,columns=['nii_path'])

In [221]:
data_json = []
for i, row in m_t1_path.iterrows():
    path1 = "/N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/" 
    path2 = m_t1_path.iloc[i][7] + "/" 
    path3 = m_t1_path.iloc[i][8] + "/"
    path4 = "/" + m_t1_path.loc[i][10] + "/"
    for j, name in enumerate(t1_names):
        path = path1 + path2 + path3 + name + path4 + "*.json"
        path_go = path1 + path2 + path3 + name + "/*" + m_t1_path.loc[i][10].split(".")[0].replace("-","").replace("_","") + "*.json"
        if glob.glob(path):
            print(path)
            data_json.append(path)
        if glob.glob(path_go):
            print(path_go)
            data_json.append(path_go)

/N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/ADNI3_MRI_fMRI_M/114_S_6039/Accelerated_Sagittal_MPRAGE/2017-07-21_12_13_00.0/*.json
/N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/ADNI3_MRI_fMRI_M/941_S_4365/Accelerated_Sagittal_MPRAGE/2017-08-28_14_06_46.0/*.json
/N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/ADNIGO_MRI_fMRI_F/002_S_1261/MPRAGE/*20110314160431*.json
/N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/ADNIGO_MRI_fMRI_F/002_S_1280/MPRAGE/*20110504132634*.json
/N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/ADNIGO_MRI_fMRI_F/002_S_2010/MPRAGE/*20100624142128*.json
/N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/ADNIGO_MRI_fMRI_F/002_S_2010/MPRAGE/*20101022153121*.json
/N/project/statadni/20231212_ADR012021_UtahBackup/ImagingData/dicomUnzipped/ADNIGO_MRI_fMRI_F/002_S_2010/MPRAGE/*20110122123155*.json
/N/proje

In [222]:
df_json = pd.DataFrame(data_json,columns=['json_path'])

In [223]:
df_nii_json = pd.concat([df_nii, df_json],axis=1)

In [224]:
df_nii_json_t1 = pd.concat([df_nii_json, m_t1],axis=1)

In [225]:
df_nii_json_t1.to_csv('missing_T1w_bidsify_paths.csv', index=False)

Here is the bash script code to copy over the data using the CSV file created in the cell above:

```while IFS=, read -r c1 c2 c3 c4 c5 c6 c7 c8; do subid="sub-ADNI${c4//_/}"; sesid=`echo ${c7} | awk -F / '{print $10}'`; if [[ -e /N/project/statadni/20250922_Saige/adni_db/bids/participants/${subid}/${sesid}/anat/ ]]; then cp $c1 /N/project/statadni/20250922_Saige/adni_db/bids/participants/${subid}/${sesid}/anat/${subid}_${sesid}_T1w.nii.gz; elif [[ ! -e /N/project/statadni/20250922_Saige/adni_db/bids/participants/${subid}/${sesid}/anat/ ]]; then mkdir /N/project/statadni/20250922_Saige/adni_db/bids/participants/${subid}/${sesid}/anat/; cp $c1 /N/project/statadni/20250922_Saige/adni_db/bids/participants/${subid}/${sesid}/anat/${subid}_${sesid}_T1w.nii.gz; fi; done < missing_T1w_bidsify_paths.csv ```

In [ ]:
# add data to tsv files? 